In [1]:
from dnallm import load_config, load_model_and_tokenizer, DNADataset, DNATrainer

In [2]:
# Load the config file
configs = load_config("./ner_task_config.yaml")

In [3]:
# Load the model and tokenizer
model_name = "zhangtaolab/plant-nucleotide-transformer-BPE"
# from Hugging Face
# model, tokenizer = load_model_and_tokenizer(model_name, task_config=configs['task'], source="huggingface")
# from ModelScope
model, tokenizer = load_model_and_tokenizer(model_name, task_config=configs['task'], source="modelscope")

Model files are stored in /Users/forrest/.cache/modelscope/hub/models/zhangtaolab/plant-nucleotide-transformer-BPE


Some weights of EsmForTokenClassification were not initialized from the model checkpoint at /Users/forrest/.cache/modelscope/hub/models/zhangtaolab/plant-nucleotide-transformer-BPE and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# Load the datasets
datasets = DNADataset.load_local_data("./rice_gene_ner_BPE.pkl", seq_col="sequence", label_col="labels", tokenizer=tokenizer, max_length=1024)

# Encode the sequences with given task's data collator
datasets.encode_sequences(task=configs['task'].task_type, remove_unused_columns=True)

# Split the dataset into train, test, and validation sets
datasets.split_data()

Format labels:   0%|          | 0/2000 [00:00<?, ? examples/s]

Encoding inputs:   0%|          | 0/2000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2067 > 2048). Running this sequence through the model will result in indexing errors


In [5]:
# Initialize the trainer
trainer = DNATrainer(
    model=model,
    config=configs,
    datasets=datasets
)

In [6]:
# Start training
metrics = trainer.train()
print(metrics)

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
500,0.570800,0.212199,0.929044,0.663283,0.603541,0.632003
1000,0.145900,0.133913,0.957369,0.780709,0.729022,0.753981


{'train_runtime': 599.7483, 'train_samples_per_second': 6.998, 'train_steps_per_second': 1.751, 'total_flos': 2381880829722624.0, 'train_loss': 0.3457653213682629, 'epoch': 3.0}


In [7]:
# Do prediction on the test set
trainer.predict()

PredictionOutput(predictions=array([[[ 6.7888454e-02, -3.2899983e-02,  5.8016468e-02, ...,
         -2.4617827e-02, -5.9096259e-01,  4.6959358e-01],
        [ 6.2947125e+00, -8.0675822e-01, -4.3537930e-01, ...,
         -1.3310690e+00, -1.8621284e+00, -1.9992840e+00],
        [ 6.1748819e+00, -7.0807642e-01, -3.2469764e-01, ...,
         -1.0625161e+00, -2.4665933e+00, -1.9149231e+00],
        ...,
        [ 6.5611745e-03, -1.6197275e+00,  6.7510307e-01, ...,
          5.3789024e+00, -2.6944914e+00, -1.0104311e+00],
        [ 1.7656660e-01, -1.9188113e+00,  4.7029319e-01, ...,
          5.4902368e+00, -2.4927297e+00, -7.0210814e-01],
        [-6.6863263e-01, -2.1539226e+00,  7.8710264e-01, ...,
          5.6398807e+00, -2.4555171e+00, -4.0269816e-01]],

       [[ 8.1358373e-01,  6.7011088e-02, -5.6578594e-01, ...,
         -1.2514906e+00, -1.0975823e+00,  8.6736917e-01],
        [ 6.2359214e+00, -5.9241629e-01, -2.6868293e-01, ...,
         -1.1766343e+00, -2.0361266e+00, -1.7637061e+0